---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def print_attributes(current):
    print(current)
    print(nx.average_shortest_path_length(current))
    print(nx.average_clustering(current))
    return

def max_calc(G):
#     import matplotlib.pyplot as plt
#     print_attributes(G)
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes( G)) for i in degree_values]
    return (np.argmax(histogram)/ len(histogram))

def graph_identification():
    res = []
    for G in P1_Graphs:
        clus_coeff = nx.average_clustering(G)
        degree_hist = max_calc(G)
        if degree_hist == 0:
            res.append('PA')
        elif clus_coeff < 0.1:
            res.append('SW_H')
        else:
            res.append('SW_L')
    return res


graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
degCent = nx.degree_centrality(G)
closeCent = nx.closeness_centrality(G, normalized=True)
betCent = nx.betweenness_centrality(G, normalized=True)    
hits = nx.hits(G)
pageRank = nx.pagerank(G, alpha=0.85)
clustering = nx.clustering(G)

In [6]:
from sklearn.metrics import recall_score, accuracy_score, precision_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler

def train_test_data():
    test_nodes = []
    train_nodes = []
    train_data = np.empty((0,10), float)
    test_data = np.empty((0,9), float)
    columns = np.array(["Node", "Degree", "DegreeCent", "Clustering", "CloseCent", "BetCent", "Hub", "Authority", "PageRank", "ManagementSalary"])
    for node in G.nodes():
        currMS = G.node[node]["ManagementSalary"]
        degree = G.degree()
        if currMS == 1 or currMS == 0: 
            train_nodes.append(node)
            train_data = np.append(train_data, np.array([[node, degree[node], degCent[node], clustering[node], closeCent[node], betCent[node], hits[0][node], hits[1][node], pageRank[node], currMS]]), axis=0)
        else: 
            test_nodes.append(node)
            test_data = np.append(test_data, np.array([[node, degree[node], degCent[node], clustering[node], closeCent[node], betCent[node], hits[0][node], hits[1][node], pageRank[node]]]), axis=0)
    train = pd.DataFrame(train_data, columns=columns)
    test = pd.DataFrame(test_data, columns=columns[:-1])
    return train, test
    
def print_stats(y_test, y_prediction):
    
#     print("accuracy  = {}".format(accuracy_score(y_test, y_prediction)))
#     print("recall    = {}".format(recall_score(y_test, y_prediction)))
#     print("precision = {}".format(precision_score(y_test, y_prediction)))
    fpr_lr, tpr_lr, _ = roc_curve(y_test, y_prediction)
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    print("auc       = {}\n".format(roc_auc_lr))
    
def salary_predictions():
    train, test = train_test_data()
    train.index = train["Node"]
    test.index = test["Node"]
    train_nodes = train["Node"]
    test_nodes = test["Node"]
    train = train.drop(["Node", "Authority", "BetCent", "PageRank"], 1)
    test = test.drop(["Node", "Authority", "BetCent", "PageRank"], 1)

    X = train.iloc[:, :-1]
    y = train.iloc[:, -1]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    test_scaled = scaler.transform(test)

    model = GradientBoostingRegressor()
    model.fit(X_train, y_train)
    y_prediction = model.predict(X_test)

#     print_stats(y_test, y_prediction)

    predicted = model.predict(test)
    
    return pd.Series(index=test_nodes,data=predicted)

salary_predictions()

Node
1.0       0.615119
2.0       0.416869
5.0       0.718385
8.0       0.453029
14.0      0.629763
18.0      0.422082
27.0      0.383363
30.0      0.454524
31.0      0.454524
34.0      0.314019
37.0      0.453029
40.0      0.383363
45.0      0.383363
54.0      0.453942
55.0      0.459372
60.0      0.454524
62.0      0.718385
65.0      0.712605
77.0      0.453029
79.0      0.383363
97.0      0.459372
101.0     0.383363
103.0     0.454524
108.0     0.383363
113.0     0.421697
122.0     0.383363
141.0     0.614734
142.0     0.623512
144.0     0.459372
145.0     0.454524
            ...   
913.0     0.454524
914.0     0.459372
915.0     0.697143
918.0     0.425609
923.0     0.425609
926.0     0.425609
931.0     0.453942
934.0     0.383363
939.0     0.383363
944.0     0.697143
945.0     0.425609
947.0     0.444190
950.0     0.695607
951.0     0.459372
953.0     0.383363
959.0     0.697143
962.0     0.697143
963.0     0.402856
968.0     0.444190
969.0     0.444190
974.0     0.434416
984.0  

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [9]:
resource_allocation = list(nx.resource_allocation_index(G))
adamic_adar_index = nx.adamic_adar_index(G)
preferential_attachment = list(nx.preferential_attachment(G))
cn_soundarajan_hopcroft = list(nx.cn_soundarajan_hopcroft(G))    

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

def new_connections_predictions():
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]
    df_cn_soundarajan_hopcroft = pd.DataFrame(index=[(x[0], x[1]) for x in cn_soundarajan_hopcroft])
    df_cn_soundarajan_hopcroft['cn_soundarajan_hopcroft'] = [x[2] for x in cn_soundarajan_hopcroft]
    df = df.join(df_cn_soundarajan_hopcroft,how='outer')
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(value=0)
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    df = future_connections.join(df,how='outer')
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
    features = ['cn_soundarajan_hopcroft', 'preferential_attachment', 'resource_allocation_index', 'jaccard_coefficient']
    X_train = df_train[features]
    Y_train = df_train['Future Connection']
    X_test = df_test[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5, random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    predictions = pd.Series(test_proba,X_test.index)
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['prob'] = [predictions[x] for x in target.index]
    return target['prob']

new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.029863
(542, 751)    0.012068
(20, 426)     0.566850
(50, 989)     0.012165
(942, 986)    0.012270
(324, 857)    0.012152
(13, 710)     0.154298
(19, 271)     0.120734
(319, 878)    0.012199
(659, 707)    0.012078
(49, 843)     0.012229
(208, 893)    0.012101
(377, 469)    0.008522
(405, 999)    0.020148
(129, 740)    0.018419
(292, 618)    0.020316
(239, 689)    0.012206
(359, 373)    0.009893
(53, 523)     0.079864
(276, 984)    0.012224
(202, 997)    0.012244
(604, 619)    0.052290
(270, 911)    0.012204
(261, 481)    0.066563
(200, 450)    0.998649
(213, 634)    0.012038
(644, 735)    0.038824
(346, 553)    0.011798
(521, 738)    0.011341
(422, 953)    0.018663
                ...   
(672, 848)    0.012204
(28, 127)     0.978425
(202, 661)    0.011757
(54, 195)     0.999850
(295, 864)    0.012137
(814, 936)    0.012053
(839, 874)    0.012270
(139, 843)    0.012145
(461, 544)    0.011026
(68, 487)     0.010971
(622, 932)    0.012113
(504, 936)    0.017265
(479, 528) 